In [1]:
import pickle
import numpy as np

NUM_NOTES = 89 # 88 + 1 for silence

X_train = pickle.load(open('data/hmm/train_probs.pkl', 'rb'))
X_test = pickle.load(open('data/hmm/test_probs.pkl', 'rb'))

Y_train = pickle.load(open('data/hmm/train_notes.pkl', 'rb'))
Y_test = pickle.load(open('data/hmm/test_notes.pkl', 'rb'))

corpus = pickle.load(open('data/tdnn/target_train.pkl', 'rb'))

In [3]:
from hmmlearn import hmm

In [4]:
# print(X_train[0:500])
print(X_train.shape)

print(corpus[0:100])

(1005186, 89)
[[False False False ..., False False  True]
 [False False False ..., False False  True]
 [False False False ..., False False  True]
 ..., 
 [False False False ..., False False  True]
 [False False False ..., False False  True]
 [False False False ..., False False  True]]


In [5]:
note_hmm = hmm.GaussianHMM(n_components=NUM_NOTES, covariance_type="full", n_iter=100)


In [6]:
count_transitions = np.zeros((NUM_NOTES, NUM_NOTES))

prev_note = 88
for note in corpus:
    count_transitions[prev_note, note] += 1
    prev_note = note
    
# Plus 1 smoothing
count_transitions = count_transitions + 0.01

In [7]:
print(count_transitions[50:53, 50:53])
# print(count_transitions.sum())

[[  1.33230100e+04   1.60100000e+01   1.00100000e+01]
 [  1.10100000e+01   3.05290100e+04   1.20100000e+01]
 [  9.01000000e+00   6.01000000e+00   1.58990100e+04]]


In [8]:
print(count_transitions[40:43, 40:43])
print(count_transitions.sum(axis=1))

[[  1.79920100e+04   1.30100000e+01   2.50100000e+01]
 [  7.01000000e+00   2.51840100e+04   1.90100000e+01]
 [  1.10100000e+01   8.01000000e+00   3.87320100e+04]]
[    424.89     530.89     546.89     535.89     542.89     539.89
     538.89     538.89     539.89     542.89     534.89     535.89
     545.89     535.89     542.89     537.89     539.89     545.89
     534.89     540.89     536.89     544.89     582.89     763.89
    1224.89    1255.89     836.89    1667.89    2168.89    2621.89
    3372.89    3120.89    9243.89    5059.89   13110.89   10355.89
   12282.89   26701.89    8494.89   27985.89   18661.89   26109.89
   39915.89   20202.89   46780.89   20025.89   43036.89   29742.89
   27422.89   44686.89   13898.89   31536.89   16484.89   19723.89
   18100.89    9072.89   13516.89    5556.89   10194.89    5797.89
    4739.89    5591.89    1839.89    3372.89    1517.89    2105.89
    1247.89    1016.89    1200.89    1019.89     577.89     654.89
     540.89     540.89     541.89

In [9]:
# Make count_transitions row-stochastic

row_sums = count_transitions.sum(axis=1)
count_transitions = count_transitions / row_sums[:, np.newaxis]

In [10]:
print(count_transitions)

[[  9.62625621e-01   2.35355033e-05   2.35355033e-05 ...,   2.35355033e-05
    2.35355033e-05   3.53267905e-02]
 [  1.88362938e-05   9.64437077e-01   1.88362938e-05 ...,   1.88362938e-05
    1.88362938e-05   3.39241651e-02]
 [  1.82852127e-05   1.84680649e-03   9.65477518e-01 ...,   1.82852127e-05
    1.82852127e-05   3.11031469e-02]
 ..., 
 [  1.83860707e-05   1.83860707e-05   1.83860707e-05 ...,   9.65287098e-01
    1.83860707e-05   3.12747063e-02]
 [  1.85566628e-05   1.85566628e-05   1.85566628e-05 ...,   1.85566628e-05
    9.64965021e-01   3.34205496e-02]
 [  4.07181267e-05   4.61435933e-05   4.88563266e-05 ...,   4.88563266e-05
    4.88563266e-05   9.48005367e-01]]


In [11]:
note_hmm.transmat_ = count_transitions

In [12]:
# Compute start probabilities as simply probability of presence of given note
start_probs = np.zeros(89)

for note_i in range(NUM_NOTES):
    
    start_probs[note_i] = len(np.argwhere(Y_train == note_i))
    
# Make start probabilities row-stochastic
row_sum = start_probs.sum()
start_probs = start_probs / row_sum

# print(start_probs)

note_hmm.startprob_ = start_probs

print(start_probs.shape)
print(len(note_hmm.startprob_))

(89,)
89


In [13]:
# Compute Gaussians for each note

means = np.zeros((NUM_NOTES, NUM_NOTES))
covs = np.zeros((NUM_NOTES, NUM_NOTES, NUM_NOTES))

for note_i in range(NUM_NOTES):
    
    probs = X_train[np.where(Y_train == note_i)]
#     print(np.where(Y_train == note_i))
    
#     print(probs[0:10])
    
    mean = np.mean(probs, axis=0)
    # print('probs shape: ' + str(probs.shape))
    cov = np.cov(probs.T)
    
    if (note_i == 1):
        
        # print("mean: " + str(mean))
        print("covariance shape: " + str(cov.shape))
        print("covariance: " + str(cov))
        
    means[note_i] = mean
    
    # Add identity here to make positive semidefinite
    covs[note_i] = cov + 0.1 * np.identity(NUM_NOTES)

covariance shape: (89, 89)
covariance: [[  9.26543507e-03   7.60514529e-03  -2.37170447e-03 ...,  -1.36334529e-05
   -1.28831965e-05  -2.40511189e-04]
 [  7.60514529e-03   1.46272743e-02   5.24101465e-04 ...,  -4.93074851e-05
   -4.55931682e-05  -2.35982825e-03]
 [ -2.37170447e-03   5.24101465e-04   2.36733799e-03 ...,  -1.98430467e-05
   -1.82276185e-05  -9.16176080e-04]
 ..., 
 [ -1.36334529e-05  -4.93074851e-05  -1.98430467e-05 ...,   5.53312475e-07
    5.09162776e-07   1.53054625e-05]
 [ -1.28831965e-05  -4.55931682e-05  -1.82276185e-05 ...,   5.09162776e-07
    4.76685943e-07   1.38495161e-05]
 [ -2.40511189e-04  -2.35982825e-03  -9.16176080e-04 ...,   1.53054625e-05
    1.38495161e-05   1.45129660e-03]]


In [14]:
note_hmm.means_ = means
note_hmm.covars_ = covs

# note_hmm.covars_ = np.tile(np.identity(NUM_NOTES), (NUM_NOTES, 1, 1))

In [15]:
pred_hidden = note_hmm.predict(X_test)

/Users/connorhargus/.virtualenvs/midi/lib/python2.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


X_test shape: (21906, 89)
X_test type: <type 'numpy.ndarray'>
float64
float64
float64


In [53]:
print(pred_hidden[3000:4000])

[88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 88 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 88 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 88 88 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46
 46 46 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 88 88 88 88 88 88 88
 88 88 88 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 52 52 52 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 88 46 46 46 46 46 46 46 46 46 46 46 46
 46 46 46 44

In [54]:
print(Y_test[3000:4000])

[88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88 88
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 88
 88 88 88 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46
 46 46 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 88 88 88 88 88 88
 88 88 88 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 51 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 46 46 46 46 46 46 46 46 46 46 46 46 46
 46 46 44 44

In [55]:
def get_accuracy(actual, pred):
    matches = 0
    for i in range(len(pred)):
        if int(pred[i]) == int(actual[i]):
            matches += 1
    
    print("Accuracy: " + str(float(matches) / len(pred_hidden)))

In [56]:
get_accuracy(Y_test, pred_hidden)

Accuracy: 0.981283666575


In [57]:
max_prob_pred = np.argmax(X_test, axis=1)

get_accuracy(Y_test, max_prob_pred)

Accuracy: 0.978362092577
